In [5]:
pip install mtcnn

     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
      --------------------------------------- 0.0/2.3 MB 1.3 MB/s eta 0:00:02
     - -------------------------------------- 0.1/2.3 MB 991.0 kB/s eta 0:00:03
     -- ------------------------------------- 0.2/2.3 MB 1.3 MB/s eta 0:00:02
     ---- ----------------------------------- 0.2/2.3 MB 1.6 MB/s eta 0:00:02
     ------- -------------------------------- 0.4/2.3 MB 1.9 MB/s eta 0:00:01
     -------- ------------------------------- 0.5/2.3 MB 1.9 MB/s eta 0:00:01
     ---------- ----------------------------- 0.6/2.3 MB 2.0 MB/s eta 0:00:01
     ------------- -------------------------- 0.8/2.3 MB 2.3 MB/s eta 0:00:01
     --------------- ------------------------ 0.9/2.3 MB 2.2 MB/s eta 0:00:01
     ------------------ --------------------- 1.1/2.3 MB 2.4 MB/s eta 0:00:01
     --------------------- ------------------ 1.2/2.3 MB 2.4 MB/s eta 0:00:01
     ---------------------- ----------------- 1.3/2.3 MB 2.4 MB/s eta

In [6]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer
from mtcnn.mtcnn import MTCNN

In [7]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'C:\Users\manoj\OneDrive\Desktop\namma project\new_model\model.h5')

In [4]:
print(eye_cascade)

< cv2.CascadeClassifier 000001F22C510ED0>


In [ ]:
#detector=MTCNN()

In [13]:
mixer.init()
sound= mixer.Sound(r'C:\Users\manoj\Downloads\file_example_WAV_1MG.wav')
capture = cv2.VideoCapture(0)
Score=0
while True:
    ret , frame = capture.read()
    height,width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    cv2.rectangle(frame, (0,height-50), (200,height), (0,0,0), thickness=cv2.FILLED)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, pt1 = (x,y), pt2 = (x+w, y+h), color = (255,0,255), thickness=2)
        
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame, pt1 = (ex,ey), pt2 = (ex+ew, ey+eh), color = (255,0,255), thickness=2)
        eye = frame[ey:ey+eh, ex:ex+ew]
        eye = cv2.resize(eye,(80,80))
        eye = eye/255
        eye = eye.reshape(80,80,3)
        eye = np.expand_dims(eye,axis=0)
        
        prediction = model.predict(eye)
        if prediction[0][0]>0.70:
            cv2.putText(frame,'closed',(10,height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,
                        color=(255,255,255),thickness=1,lineType=cv2.LINE_AA)
            cv2.putText(frame,'score:'+str(Score),(100,height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,
                        color=(255,255,255),thickness=1)
            Score=Score+1
            if(Score>15):
                try:
                    sound.play()
                except:
                    pass
        
        elif prediction[0][1]>0.50:
            cv2.putText(frame,'open',(10,height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,
                        color=(255,255,255),thickness=1)
            cv2.putText(frame,'score:'+str(Score),(100,height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,
                        color=(255,255,255),thickness=1)
            Score = Score-1
            if (Score<0):
                Score=0
    cv2.imshow('frame',frame)
    if cv2.waitKey(20) & 0xFF==ord('q'):
        break
        
capture.release()
cv2.destroyAllWindows()